ESTANDARIZACIÓN

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import researchpy as rp
import math

from scipy import stats
import researchpy as rp
from scipy.stats import levene
from scipy.stats import chi2_contingency
from sklearn.preprocessing import RobustScaler


plt.rcParams["figure.figsize"] = (10,8)
pd.options.display.max_columns = None

import warnings
warnings.filterwarnings('ignore')

In [21]:
df = pd.read_pickle('../data/bikes_limpieza_sin_totales.pkl')
df.head(2)

,fecha,estacion,año,mes,festivo,dia,laborable,meteo,temperatura,sensacion_termica,humedad,velo_viento,usu_casuales,usu_registrados
0,2018-01-01,invierno,2018,ene,festivo,lun,no laborable,nublado,14.110847,18.18125,80.5833,10.749882,331,654
1,2018-01-02,invierno,2018,ene,no festivo,mar,laborable,nublado,14.902598,17.68695,69.6087,16.652113,131,670


Vamos a eliminar la VP numérica "temperatura" que vimos en visualización que correlacionaba muy alto (0.99) con sensación térmica.
Vamos a eliminar de nuestro modelo predictivo la V "usu_casuales" porque aquí no aplica como VP.

In [22]:
df.drop(["temperatura", "usu_casuales"], axis= 1, inplace= True)

In [23]:
df.head()

,fecha,estacion,año,mes,festivo,dia,laborable,meteo,sensacion_termica,humedad,velo_viento,usu_registrados
0,2018-01-01,invierno,2018,ene,festivo,lun,no laborable,nublado,18.18125,80.5833,10.749882,654
1,2018-01-02,invierno,2018,ene,no festivo,mar,laborable,nublado,17.68695,69.6087,16.652113,670
2,2018-01-03,invierno,2018,ene,no festivo,mie,laborable,buen tiempo,9.47025,43.7273,16.636703,1229
3,2018-01-04,invierno,2018,ene,no festivo,jue,laborable,buen tiempo,10.60610,59.0435,10.739832,1454
4,2018-01-05,invierno,2018,ene,no festivo,vie,laborable,buen tiempo,11.46350,43.6957,12.522300,1518


In [24]:
numericas = df.select_dtypes(include = np.number).drop("usu_registrados", axis= 1)
numericas.head(2)

,sensacion_termica,humedad,velo_viento
0,18.18125,80.5833,10.749882
1,17.68695,69.6087,16.652113


In [25]:
robust = RobustScaler()
robust.fit(numericas)
X_robust = robust.transform(numericas)
numericas_robust = pd.DataFrame(X_robust, columns = numericas.columns)
numericas_robust.head(2)

,sensacion_termica,humedad,velo_viento
0,-0.456399,0.858854,-0.208909
1,-0.492863,0.331548,0.687550


In [26]:
df_robust = df.copy()
df_robust[numericas_robust.columns] = numericas_robust


In [27]:
df_robust.head(2)

,fecha,estacion,año,mes,festivo,dia,laborable,meteo,sensacion_termica,humedad,velo_viento,usu_registrados
0,2018-01-01,invierno,2018,ene,festivo,lun,no laborable,nublado,-0.456399,0.858854,-0.208909,654
1,2018-01-02,invierno,2018,ene,no festivo,mar,laborable,nublado,-0.492863,0.331548,0.687550,670


In [28]:
# Guardamos el df normalizado:

df_robust.to_pickle('../data/bikes_registrados_estand.pkl')